In [15]:
import numpy as np
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import re
from bs4 import BeautifulSoup

from  googleapiclient.discovery import  build
from googleapiclient.errors import HttpError

from textblob import TextBlob

porter=PorterStemmer()

In [4]:
api_key='YOUR_API_KEY'

def video_comments(video_id):
    li = []
    # empty list for storing reply
    replies = []

    # creating youtube resource object
    youtube = build('youtube', 'v3',
                developerKey=api_key)

    # retrieve youtube video results
    video_response=youtube.commentThreads().list(
    part='snippet',
    textFormat='plainText',
    videoId=video_id,
    maxResults=100
    ).execute()

    # iterate video response
    while video_response:

    # extracting required info
    # from each result object
        for item in video_response['items']:

        # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            li.append(comment)

    # Again repeat
        next_page_token = video_response.get('nextPageToken')
        if next_page_token:
          video_response = youtube.commentThreads().list(
                part='snippet',
                textFormat='plainText',
                videoId=video_id,
                maxResults=100,
                pageToken=next_page_token
            ).execute()
        else:
           break
    return(li)

comments=video_comments('eMlx5fFNoYc')
temp=video_comments('eMlx5fFNoYc')

In [5]:
len(comments)

1000

In [153]:
data=pd.DataFrame(comments,columns=['comments'])

In [154]:
data['comments']

0      A few added notes based on common comments I s...
1                      I have wait 3 months for Chapter7
2      6:55 can the query ask forwards also? like let...
3                                Can you do video on gan
4                           Can’t wait for the next one?
                             ...                        
995                                      14 second ago?!
996                                                first
997                                                FIRST
998                                                    ❤
999                                          First here😮
Name: comments, Length: 1000, dtype: object

In [155]:
def stemming_words(text):
    review=text.split()
    review=[porter.stem(word) for word in review]
    review=" ".join(review)
    print(review)
    return review

In [157]:
data['comments']=data['comments'].apply(lambda x:re.sub('[^a-z A-Z 0-9-]','',x))
#lowering case
data['comments']=data['comments'].apply(lambda x:x.lower())
#Removing stopwords
data['comments']=data['comments'].apply(lambda x:' '.join([y for y in x.split() if y not in stopwords.words('english')]))
#Removing url
data['comments']=data['comments'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
#Removing html tags
data['comments']=data['comments'].apply(lambda x: BeautifulSoup(x,"lxml").get_text())
#Removng extra spaces
data['comments']=data['comments'].apply(lambda x:" ".join(x.split()))
#Performing lemmatization
data['comments']=data['comments'].apply(lambda x: stemming_words(x))

ad note base common comment seeconcern mask self-att sever peopl ask case feel like later word updat mean earlier word like languag adject follow noun model alway put richest mean last token eg earli noun get bake later adject exampl victorlevoso8984 note empir evid suggest mean sentenc often get bake emb punctuat mark end keep mind model doesnt conceptu thing way human likelihood doesnt wouldnt over-index motiv exampl given videoalso one thing call explicitli person like think vector like emb key queri etc column convent display way sourc includ attent need paper may present organ row-by-row fashion relev par equat shown 1029 express paper look like q kt would convent video instead look like kt q
wait 3 month chapter7
655 queri ask forward also like let say saw creatur huge foul eat grass level produc similar result huge foul creatur saw eat grass way theyd seem similar creatur queri forward backward
video gan
cant wait next one
wish explicitli mention step ad comput v vector origin i

In [158]:
comments_cleaned=data['comments'].tolist()

In [159]:
df=pd.DataFrame(columns=['Sentiment_Score','Nature of comment'])
def get_sentiment_scores(comments):
    positive_scores = []
    negative_scores = []
    neutral_scores=[]
    for comment in comments:
        info=[]
        blob = TextBlob(comment)
        sentiment_score = blob.sentiment.polarity
        info.append(sentiment_score)
        if sentiment_score > 0:
            positive_scores.append(sentiment_score)
            info.append("positive")
        elif sentiment_score < 0:
            negative_scores.append(sentiment_score)
            info.append("negative")
        elif sentiment_score == 0:
            neutral_scores.append(sentiment_score)
            info.append("neutral")
        df.loc[len(df)]= info


    return positive_scores,neutral_scores, negative_scores

In [160]:
positive_scores,neutral_scores, negative_scores=get_sentiment_scores(comments_cleaned)
pos=np.mean(positive_scores)
neg=np.mean(negative_scores)
neu=np.mean(neutral_scores)
len(positive_scores),len(negative_scores),len(neutral_scores)

(384, 116, 500)

In [161]:
df.head()

,Sentiment_Score,Nature of comment
0,-0.169792,negative
1,0.000000,neutral
2,0.200000,positive
3,0.000000,neutral
4,0.000000,neutral


In [163]:
df.insert(0,
          column="Comments",
          value=temp)

In [164]:
df.head()

,Comments,Sentiment_Score,Nature of comment
0,A few added notes based on common comments I s...,-0.169792,negative
1,I have wait 3 months for Chapter7,0.000000,neutral
2,6:55 can the query ask forwards also? like let...,0.200000,positive
3,Can you do video on gan,0.000000,neutral
4,Can’t wait for the next one?,0.000000,neutral
